Реализуйте алгоритм SAC для среды lunar lander

In [1]:
!pip install swig
!pip install "gymnasium[box2d]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp311-cp311-linux_x86_64.whl size=2379367 sha256=af55ee9ca8a0a390ba0f3103a493f2dcdeaa3bb2539bfcfb56f264953358b523
  Stored in directory: /root/.cache/pip/wheels/ab/f1/0c/d56f4a2bdd12bae0a0693ec33f2f0daadb5eb9753c78fa5308
Successfully built box2d-py


In [2]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
import random
from torch.distributions import Normal

In [3]:
GAMMA = 0.99
TAU = 0.005
ALPHA = 0.2
ACTOR_LR = 3e-4
CRITIC_LR = 3e-4
REPLAY_SIZE = 100000
BATCH_SIZE = 256
START_STEPS = 10000
TOTAL_STEPS = 200000
UPDATE_AFTER = 1000
UPDATE_EVERY = 50

In [ ]:
class Actor(nn.Module):
    def __init__(self, obs_dim, act_dim, act_limit):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_dim, 256), nn.ReLU(),
            nn.Linear(256, 256), nn.ReLU(),
        )
        self.mu_layer = nn.Linear(256, act_dim)
        self.log_std_layer = nn.Linear(256, act_dim)
        self.act_limit = act_limit

    def forward(self, obs):
        x = F.relu(self.net(obs))
        mean, std = self.mu_layer(x),  torch.clamp(self.log_std_layer(x), -20, 2).exp()
        normal = torch.distributions.Normal(mean, std)

        x_t = normal.rsample()
        y_t = torch.tanh(x_t)
        action = y_t * (action_high - action_low) / 2.0 + (action_low + action_high) / 2.0

        log_prob = normal.log_prob(x_t)
        log_prob -= torch.log((1 - y_t.pow(2)) + 1e-6)
        log_prob = log_prob.sum(1, keepdim=True)

        return action, log_prob

  # Напишите функцию, которая возвращает только действие. Если deterministic True, то действие не семплируется, а просто берётся mean (его всё ещё надо предобразовать)
    def get_action(self, obs, deterministic=False):
        mu = self.mu_layer(self.net(obs))

        if deterministic:
              action = torch.tanh(mu) * self.act_limit
        else:
            with torch.no_grad():
              std = torch.clamp(self.log_std_layer(self.net(obs)), -20, 2).exp()
              normal = Normal(mu, std)
              u = normal.rsample()
              action = torch.tanh(u) * self.act_limit
        return action.cpu().numpy()




In [19]:
class Critic(nn.Module):
    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.q1 = nn.Sequential(
            nn.Linear(obs_dim + act_dim, 256), nn.ReLU(),
            nn.Linear(256, 256), nn.ReLU(),
            nn.Linear(256, 1)
        )
        self.q2 = nn.Sequential(
            nn.Linear(obs_dim + act_dim, 256), nn.ReLU(),
            nn.Linear(256, 256), nn.ReLU(),
            nn.Linear(256, 1)
        )

    def forward(self, obs, act):
        x = torch.cat([obs, act], dim=-1)
        return self.q1(x), self.q2(x)

In [20]:
class ReplayBuffer:
    def __init__(self, size):
        self.buffer = deque(maxlen=size)

    def add(self, *args):
        self.buffer.append(tuple(args))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = map(np.array, zip(*batch))
        return (
            torch.tensor(states, dtype=torch.float32),
            torch.tensor(actions, dtype=torch.float32),
            torch.tensor(rewards, dtype=torch.float32).unsqueeze(1),
            torch.tensor(next_states, dtype=torch.float32),
            torch.tensor(dones, dtype=torch.float32).unsqueeze(1)
        )

In [26]:
env = gym.make("LunarLanderContinuous-v3")
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]
action_low, action_high = float(env.action_space.low[0]), float(env.action_space.high[0])

act_limit  = (action_high - action_low) / 2


actor = Actor(obs_dim, act_dim, act_limit)
critic = Critic(obs_dim, act_dim)
critic_target = Critic(obs_dim, act_dim)
critic_target.load_state_dict(critic.state_dict())

actor_optim = optim.Adam(actor.parameters(), lr=ACTOR_LR)
critic_optim = optim.Adam(critic.parameters(), lr=CRITIC_LR)

replay_buffer = ReplayBuffer(REPLAY_SIZE)

obs, _ = env.reset()
episode_return, episode_len = 0, 0

In [27]:
def update():
    if REPLAY_SIZE < BATCH_SIZE:
        return

    state, action, reward, next_state, done = replay_buffer.sample(BATCH_SIZE)

    with torch.no_grad():
      next_action, next_log_prob = actor(next_state)
      q1_target, q2_target = critic_target(next_state, next_action)
      q_target = torch.min(q1_target, q2_target) - ALPHA * next_log_prob
      target = reward + (1 - done) * GAMMA * q_target

    q1, q2 = critic(state, action)
    critic_loss = F.mse_loss(q1, target) + F.mse_loss(q2, target)

    critic_optim.zero_grad()
    critic_loss.backward()
    critic_optim.step()

    new_action, new_log_prob = actor(state)
    q1, q2 =  critic(state, new_action)
    actor_loss = (ALPHA * new_log_prob - torch.min(q1, q2)).mean()

    actor_optim.zero_grad()
    actor_loss.backward()
    actor_optim.step()

    for param, target_param in zip(critic.parameters(), critic_target.parameters()):
        target_param.data.copy_(TAU * param.data + (1 - TAU) * target_param.data)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
actor.to(device)
critic.to(device)
critic_target.to(device)


for step in range(TOTAL_STEPS):
    if step < START_STEPS:
        act = env.action_space.sample()
    else:
        with torch.no_grad():
            obs_t = torch.tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)
            act = actor.get_action(obs_t).detach().numpy()[0]

    next_obs, rew, terminated, truncated, _ = env.step(act)
    done = terminated or truncated
    replay_buffer.add(obs, act, rew, next_obs, done)

    obs = next_obs
    episode_return += rew
    episode_len += 1

    if done:
        obs, _ = env.reset()
        print(f"Step: {step}, Return: {episode_return:.2f}, Len: {episode_len}")
        episode_return, episode_len = 0, 0

    if step >= UPDATE_AFTER and step % UPDATE_EVERY == 0:
        for _ in range(UPDATE_EVERY):
            update()

Step: 49, Return: -274.43, Len: 187
Step: 188, Return: -186.41, Len: 139
Step: 306, Return: -274.34, Len: 118
Step: 386, Return: -60.32, Len: 80
Step: 496, Return: -349.35, Len: 110
Step: 600, Return: -87.94, Len: 104
Step: 723, Return: -235.87, Len: 123
Step: 817, Return: -72.80, Len: 94
Step: 936, Return: -407.68, Len: 119
Step: 1058, Return: -91.40, Len: 122
Step: 1163, Return: -439.44, Len: 105
Step: 1236, Return: -193.69, Len: 73
Step: 1339, Return: -460.69, Len: 103
Step: 1468, Return: -225.29, Len: 129
Step: 1559, Return: -300.67, Len: 91


KeyboardInterrupt: 